# Estimate differences in modeled and remotely-sensed SMB

1. Snowline altitudes (SLAs)
2. Equilibrium line altitudes (ELAs)
3. Modeled surface mass balance (SMB) at the snowline

In [ ]:
import os,glob
import xarray as xr
import matplotlib.pyplot as plt
import geopandas as gpd
import pandas as pd
from tqdm.auto import tqdm
import numpy as np

# Define paths to data
smb_mod_dir = '/Volumes/LaCie/raineyaberle/Research/PhD/snow_cover_mapping/Rounce_et_al_2023/glac_SMB_binned'
scs_dir = '/Volumes/LaCie/raineyaberle/Research/PhD/snow_cover_mapping/study-sites'

## 1. Snowline altitudes

## Compile modeled melt factors

In [ ]:
# Check if already exists in file
fsnow_mod_fn = '/Volumes/LaCie/raineyaberle/Research/PhD/snow_cover_mapping/analysis/fsnow_mod.csv'
if not os.path.exists(fsnow_mod_fn):
    print('Compiling modeled melt factors of snow')
    # Load AOIs for RGI IDs
    aois_fn = '/Volumes/LaCie/raineyaberle/Research/PhD/snow_cover_mapping/analysis/all_aois.shp'
    aois = gpd.read_file(aois_fn)
    modelprm_dir = '/Volumes/LaCie/raineyaberle/Research/PhD/snow_cover_mapping/Rounce_et_al_2023/modelprms'
    # Initialize dataframe
    fsnow_mod = pd.DataFrame()
    # Iterate over RGI IDs
    for rgi_id in tqdm(aois['RGIId'].drop_duplicates().values):
        # Load model parameters
        modelprm_fn = os.path.join(modelprm_dir, f"{rgi_id.replace('RGI60-0','')}-modelprms_dict.pkl")
        modelprm = pd.read_pickle(modelprm_fn)
        # Take the median of MCMC fsnow results (not much different than the mean)
        ddfsnow_mcmc = np.array(modelprm['MCMC']['ddfsnow']['chain_0'])
        df = pd.DataFrame({"RGIId": [rgi_id],
                           "fsnow": [np.median(ddfsnow_mcmc)]})
        # Concatenate df to full dataframe
        fsnow_mod = pd.concat([fsnow_mod, df])
    # Save to file
    fsnow_mod.reset_index(drop=True, inplace=True)
    fsnow_mod.to_csv(fsnow_mod_fn, index=False)
    print('Compiled melt factors of snow saved to file:', fsnow_mod_fn)

else:
    fsnow_mod = pd.read_csv(fsnow_mod_fn)
    print('Compiled melt factors of snow loaded from file.')

plt.hist(fsnow_mod['fsnow'], bins=50)
plt.show()



## Estimate observed melt factors of snow

$\Sigma Accumulation - \Sigma PDDs * f_{snow} = SMB$

At the snowline, SMB = 0. Thus, the approach is to use modeled accumulation and temperature (PDD) time series at the snowline altitude from ERA5 to solve for $f_{snow}$.

In [ ]:
# Grab climate and modeled SMB file names
temp_dir = '/Volumes/LaCie/raineyaberle/Research/PhD/snow_cover_mapping/Rounce_et_al_2023/glac_temp_monthly'
temp_fns = sorted(glob.glob(os.path.join(temp_dir, 'R*.nc')))
accum_dir = '/Volumes/LaCie/raineyaberle/Research/PhD/snow_cover_mapping/Rounce_et_al_2023/glac_accum_monthly'
accum_fns = sorted(glob.glob(os.path.join(accum_dir, 'R*.nc')))
smb_dir = '/Volumes/LaCie/raineyaberle/Research/PhD/snow_cover_mapping/Rounce_et_al_2023/glac_SMB_binned'
smb_fns = sorted(glob.glob(os.path.join(smb_dir, '*.nc')))

# Define path to snow cover stats
scs_path = '/Volumes/LaCie/raineyaberle/Research/PhD/snow_cover_mapping/study-sites'

# Iterate over RGI IDs
for rgi_id in aois['RGIId'].drop_duplicates().values[0:10]:
    # temp and accum
    if "-01." in rgi_id:
        region = "R01"
    elif "-02." in rgi_id:
        region = "R02"
    rng = rgi_id.split('.')[1]

    print(rgi_id, region, rng)
# smb_fn = smb_fns[0]
# smb = xr.open_dataset(smb_fn)
# smb

In [ ]:
smb_fn = smb_fns[0]
smb = xr.open_dataset(smb_fn)
smb